In [1]:
import zipfile
import os

# Change this path if you upload a new ZIP file
zip_path =r"C:\Users\manas\Downloads\archive (25).zip"  # Update with new ZIP file path
extracted_path = r"C:\Users\manas\Downloads\archive (25)"
# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

print(f"Dataset extracted to: {extracted_path}")

Dataset extracted to: C:\Users\manas\Downloads\archive (25)


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Directory structure setup
train_dir = os.path.join(extracted_path, 'train')
test_dir = os.path.join(extracted_path, 'test')

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Organize into train-test split
folders = os.listdir(extracted_path)
for folder in folders:
    folder_path = os.path.join(extracted_path, folder)

    if not os.path.isdir(folder_path):
        continue  # Skip non-folder files

    images = os.listdir(folder_path)
    if len(images) == 0:
        continue  # Skip empty folders

    # Split images into train and test
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Create subdirectories for each folder in train and test
    train_subdir = os.path.join(train_dir, folder)
    test_subdir = os.path.join(test_dir, folder)

    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

    # Move image files (not directories)
    for image in train_images:
        image_path = os.path.join(folder_path, image)
        if os.path.isfile(image_path):  # Check if it's a file
            shutil.move(image_path, os.path.join(train_subdir, image))

    for image in test_images:
        image_path = os.path.join(folder_path, image)
        if os.path.isfile(image_path):  # Check if it's a file
            shutil.move(image_path, os.path.join(test_subdir, image))

print("Dataset organized into train and test folders.")

Dataset organized into train and test folders.


In [3]:
import cv2
import imutils
from tqdm import tqdm

# Function to crop images
def crop_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    try:
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = 0
        cropped_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS]
        return cropped_img
    except Exception as e:
        print(f"Error cropping image: {e}")
        return img

# Apply cropping
for dataset_type in ['train', 'test']:
    dataset_path = os.path.join(extracted_path, dataset_type)

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if not os.path.isdir(folder_path):
            continue

        for img_name in tqdm(os.listdir(folder_path), desc=f"Processing {folder} in {dataset_type}"):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)

            if img is not None:
                cropped_img = crop_img(img)
                cv2.imwrite(img_path, cropped_img)

Processing Sport Utility Vehicle in train: 100%|██████████████████████████████████| 1460/1460 [00:02<00:00, 641.22it/s]
Processing test in train: 0it [00:00, ?it/s]
Processing train in train: 0it [00:00, ?it/s]
Processing Sport Utility Vehicle in test: 100%|█████████████████████████████████████| 366/366 [00:00<00:00, 565.21it/s]
Processing test in test: 0it [00:00, ?it/s]
Processing train in test: 0it [00:00, ?it/s]
Processing Van in test: 100%|███████████████████████████████████████████████████████| 434/434 [00:00<00:00, 578.16it/s]


In [10]:
# Install YOLOv8 first
!pip install ultralytics


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os
from ultralytics import YOLO

# Define paths for training and validation data
train_dir =r"C:\Users\manas\Downloads\archive (25)\train"  # Path to your training data
test_dir = r"C:\Users\manas\Downloads\archive (25)\test"   # Path to your validation data

# Define the YOLOv8 Nano model
model = YOLO("yolov8n.yaml")  # Using the Nano version for quick training

# Define the YAML configuration for the dataset
data_yaml = f"""
train: {train_dir}  # Path to training data
val: {test_dir}    # Path to validation data
nc: 7  # Number of classes
names: ['Big Truck', 'City Car', 'Multi Purpose Vehicle', 'Sedan', 'Sport Utility Vehicle', 'Truck', 'Van']  # Class names
"""

# Save the YAML configuration to the dataset folder
extracted_path =r"C:\Users\manas\Downloads\archive (25)"  # Path to the extracted dataset
yaml_path = os.path.join(extracted_path, "data.yaml")

# Create the YAML file with the correct paths
with open(yaml_path, "w") as f:
    f.write(data_yaml)

print(f"Dataset YAML saved at: {yaml_path}")

# Train the YOLO model (force to use CPU)
model.train(data=yaml_path, epochs=2, imgsz=640, device='cpu')  # Use 'device=cpu' if no GPU is available


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

Dataset YAML saved at: C:\Users\manas\Downloads\archive (25)\data.yaml


New https://pypi.org/project/ultralytics/8.3.33 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.0+cpu CPU (11th Gen Intel Core(TM) i5-11260H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=C:\Users\manas\Downloads\archive (25)\data.yaml, epochs=2, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None,

zero-size array to reduction operation maximum which has no identity


Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        1/2         0G          0      67.18          0          0        640: 100%|██████████| 783/783 [2:13:26<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [04:28
                   all       3132          0          0          0          0          0
WARNING  no labels found in detect set, can not compute metrics without labels

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        2/2         0G          0       8.54          0          0        640: 100%|██████████| 783/783 [7:12:49<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [04:44
                   all       3132          0          0          

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: []
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000015E0B534A10>
fitness: 0.0
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([], dtype=float64)
names: {0: 'Big Truck', 1: 'City Car', 2: 'Multi Purpose Vehicle', 3: 'Sedan', 4: 'Sport Utility Vehicle', 5: 'Truck', 6: 'Van'}
plot: True
results_dict: {'metrics/precision(B)': 0.0, 'metrics/recall(B)': 0.0, 'metrics/mAP50(B)': 0.0, 'metrics/mAP50-95(B)': 0.0, 'fitness': 0.0}
save_dir: WindowsPath('runs/detect/train')
speed: {'preprocess': 1.1102393516940023, 'inference': 64.65179009760324, 'loss': 0.0, 'postprocess': 0.3140744578335934}

In [ ]:
import cv2
from ultralytics import YOLO

# Load the trained model
trained_model = YOLO(r"C:\Users\manas\Downloads\runs\detect\train\weights\best.pt")

# Test the model on a new image
test_image_path = r"C:\Users\manas\Downloads\archive (25)\test\Truck"  # Update with the test image path
results = trained_model(test_image_path)

# Show the results for each image in the list
for result in results:  # Iterate over the list of results
    # Use result.plot() to get the annotated image as a numpy array
    annotated_image = result.plot()

    # Display the annotated image using OpenCV
    cv2.imshow("Annotated Image", annotated_image)
    cv2.waitKey(0)  # Wait for a key press to close the image
    cv2.destroyAllWindows()


image 1/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013194.jpg: 640x544 48 Big Trucks, 4 Sedans, 29 Sport Utility Vehicles, 14 Trucks, 68.9ms
image 2/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013201.jpg: 480x640 (no detections), 59.7ms
image 3/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013214.jpg: 640x576 (no detections), 70.4ms
image 4/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013221.jpg: 640x512 (no detections), 66.7ms
image 5/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013222.jpg: 640x640 (no detections), 81.0ms
image 6/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013230.jpg: 640x640 26 Big Trucks, 3 Sedans, 24 Sport Utility Vehicles, 9 Trucks, 79.3ms
image 7/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013235.jpg: 640x640 (no detections), 79.6ms
image 8/230 C:\Users\manas\Downloads\archive (25)\test\Truck\Image_013240.jpg: 640x640 (no detections), 73.8ms
image 9/230 C:\Users

In [ ]:
# Evaluate the model on the test data
metrics = trained_model.val()
print(metrics)